In [28]:
import redis 

client = redis.Redis('host.docker.internal', 6379, 0)

client.flushdb()

True

In [29]:
import json
import numpy as np
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from pcomp.kafka_handlers import KafkaProducerHandler, KafkaConsumerHandler, KafkaConsumerHandlerNeuron
from pcomp.activation_functions import ACTIVATIONS, relu, softmax
from pcomp.redis_utils import RedisHandler
from pcomp.parser import parse_layer_coordinator_message, parse_layer_message
from pcomp.avro_utils import avro_serialize, avro_deserialize
from pcomp.neurons_accumulator import NeuronsAccumulator

# Kafka Configuration
KAFKA_BROKER = 'kafka:29092'

In [30]:
class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation = activation
        self.activation_func = ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 5)
        self.producer = None
        self.executor = ThreadPoolExecutor(max_workers=4)
        

    def fetch_input(self, image_id):
        key = f"streams:{image_id}:initial_data" if self.layer_id == 'layer_0' else f"streams:{image_id}:{self.layer_id_num - 1}"
        # Poll Redis until the data is available.
        while True:
            data = np.frombuffer(self.redis_handler.get(key), dtype=np.float64)
            if data is not None:
                return data
            print(f"⏳ Neuron {self.neuron_id} waiting for input data for key: {key}")
            time.sleep(0.5)

    def process_and_send(self, image_id, input_data):
        z = np.dot(input_data, self.weights) + self.bias
        output = z if self.is_final_layer else self.activation_func(z)
        #key = f"outputs:{image_id}:{self.layer_id_num}"
        #self.redis_handler.store_neuron_output(key, self.neuron_id, output)
        msg = f"{image_id}|{self.neuron_id}|{format(output, '.17g')}"
        self.producer.send(msg)
        #self.producer.send(f'requests-responses', 'www.neuron.example')

    def run(self):
        # Instantiate Kafka consumer and producer inside the thread.
        consumer = KafkaConsumerHandler(f'layer-{self.layer_id_num}', KAFKA_BROKER, group_id=f"{self.neuron_id}_{self.layer_id_num}_group")
        self.producer = KafkaProducerHandler(KAFKA_BROKER, f'layer-{self.layer_id_num}-complete')
        last_msg_time = time.time()
        while True:
            got_message = False
            for message in consumer.consume():
                got_message = True
                last_msg_time = time.time()
                image_id_str = message
                image_id = int(image_id_str)
                input_data = self.fetch_input(image_id)
                self.process_and_send(image_id, input_data)
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                self.producer.close()
                break


class LayerCoordinator(threading.Thread):
    def __init__(self, layer_id, neuron_count, is_final_layer=False):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_count = neuron_count
        self.is_final_layer = is_final_layer
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 40)
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.producer = None

    def run(self):
        consumer = KafkaConsumerHandler(f'layer-{self.layer_id_num}-complete', KAFKA_BROKER, group_id=f"{self.layer_id_num}_coord_group")
        if not self.is_final_layer:
            self.producer = KafkaProducerHandler(KAFKA_BROKER, f"layer-{self.layer_id_num + 1}")
        last_msg_time = time.time()
        while True:
            got_message = False
            for message in consumer.consume():
                got_message = True
                last_msg_time = time.time()
                image_id_str, neuron_id_str, output = message.split("|")
                image_id = int(image_id_str)
                neuron_id = int(neuron_id_str)
                key = f"streams:{image_id}:outputs:{self.layer_id_num}"
                cnt_key = f"streams:{image_id}:cnt:{self.layer_id_num}"
                cnt = self.redis_handler.store_neuron_result(key, cnt_key, neuron_id, output, self.neuron_count)
                if cnt == self.neuron_count:
                    self.aggregate_neuron_outputs(image_id, key)
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                if self.producer:
                    self.producer.close()
                break

    def aggregate_neuron_outputs(self, image_id, key):
        outputs = self.redis_handler.get_output_vector(key, self.neuron_count)
        if not self.is_final_layer:
            self.redis_handler.set(f"streams:{image_id}:{self.layer_id_num}", outputs, True, 60)
            msg = f"{image_id}"
            self.producer.send(msg)
        else:
            prediction = int(np.argmax(outputs))
            self.redis_handler.hset('streams:predictions', image_id, prediction)
            self.redis_handler.delete_streams_keys(image_id)

# Load network and dataset
data = json.load(open("node_based_model.json"))
#df = pd.read_csv('data/mnist.csv').head(10)

neurons = []
layers = []
coordinators = []

for layer_name, layer_info in data.items():
    neurons += [Neuron(layer_id=layer_name, neuron_id=i, weights=node['weights'], bias=node['biases'], activation=node['activation'], is_final_layer=(layer_name == list(data.keys())[-1])) for i, node in enumerate(layer_info['nodes'])]
    coordinators.append(LayerCoordinator(layer_id=layer_name, neuron_count=len(layer_info['nodes']), is_final_layer=(layer_name == list(data.keys())[-1])))

# Start all threads
for thread in neurons + coordinators:
    thread.start()

print("Threads started")

Threads started


In [32]:
# Wait for all threads to complete
for thread in neurons + coordinators:
    thread.join()

print("Threads finished")

Threads finished


In [31]:
import redis

# Connect to Redis
r = RedisHandler('host.docker.internal', 6379, 0)

# Get hashes from Redis
images_label = r.hgetall('streams:images_label')
predictions = r.hgetall('streams:predictions')

# Decode bytes to string
images_label = {k.decode(): v.decode() for k, v in images_label.items()}
predictions = {k.decode(): v.decode() for k, v in predictions.items()}

# Calculate accuracy
correct = 0
total = len(images_label)

for field, label_val in images_label.items():
    pred_val = predictions.get(field, None)
    if pred_val == label_val:
        correct += 1

accuracy = (correct / total) * 100 if total > 0 else 0

print(f'Accuracy: {accuracy:.2f}% ({correct}/{total})')


Accuracy: 96.20% (962/1000)


In [37]:
r = redis.Redis('host.docker.internal', 6379, 0)
l = r.hlen("outputs:1:0")
print(l)

128


In [12]:
r = RedisHandler('host.docker.internal', 6379, 0)

# lua_script = """
# redis.call('HSET', KEYS[1], ARGV[1], ARGV[2])
# local incrResult = redis.call('INCR', KEYS[2])
# return incrResult
# """

# redis_handler = RedisHandler('host.docker.internal', 6379, 0)

result = r.store_neuron_result("myhash", "counter", "field3", "value")

print("Result of INCR:", result)

Result of INCR: 10


In [16]:
import redis

# Connect to Redis
r = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Lua script that pushes all values from a hash into a list and returns the list
lua_script = """
local hashKey = KEYS[1]
local listKey = KEYS[2]

local fields = redis.call("HKEYS", hashKey)
for i, field in ipairs(fields) do
    local value = redis.call("HGET", hashKey, field)
    redis.call("RPUSH", listKey, value)
end
return redis.call("LRANGE", listKey, 0, -1)
"""

# Number 2 indicates that there are two keys: the hash key and the list key.
result = r.eval(lua_script, 2, "myhash", "mylist")

print("Result:", result)

Result: []
